# 의사록 hwp 파일 크롤링 (final)

In [3]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def get_hwp_data(startyear, startmonth, endyear, endmonth):
    driver = webdriver.Chrome()
    
    result = []

    try:
        while startyear <= endyear:
            url = f'https://www.bok.or.kr/portal/singl/crncyPolicyDrcMtg/listYear.do?mtgSe=A&menuNo=200755&pYear={startyear}#content'
            driver.get(url)
            time.sleep(3)
            
            tr_tags = driver.find_elements(By.CSS_SELECTOR, '#tableId > tbody > tr')
            
            for tr_tag in tr_tags:
                day = tr_tag.find_element(By.CSS_SELECTOR,'th').text
                month = int(day[:2])

                if startyear == 2005 and month < startmonth:
                    continue
                if endyear == 2012 and month > endmonth:
                    break

                td_tag = tr_tag.find_elements(By.CSS_SELECTOR,'td')[2]
                if startyear == 2006 or (startyear == 2007) & (month == 1):
                    li_tag = td_tag.find_elements(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li')[1]
                    a_tag = li_tag.find_element(By.CSS_SELECTOR,'a')
                    dlink = a_tag.get_attribute('href')
                else:
                    a_tag = td_tag.find_element(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li>a')
                    dlink = a_tag.get_attribute('href')

                result.append(dict(
                    year = startyear,
                    day = day,
                    dlink = dlink
                ))

            startyear += 1
            
    finally:
        driver.quit()

    return result

def convert_to_datetime(df):
    # 'day' 컬럼에서 "월"과 "일" 부분 제거
    df['day'] = df['day'].str.replace('월', '').str.replace('일', '').str.split('(').str[0].str.strip()
    
    # 'year'와 'day'를 합쳐서 새로운 컬럼 'date' 생성
    df['date'] = df['year'].astype(str) + ' ' + df['day']
    
    # 'date' 컬럼을 datetime 타입으로 변환
    df['date'] = pd.to_datetime(df['date'], format='%Y %m %d')
    
    # 'year'와 'day' 컬럼 제거
    df = df.drop(columns=['year', 'day'])
    
    # 열 순서를 변경하여 'date'가 먼저 오도록 설정
    df = df[['date', 'dlink']]
    
    return df

data = get_hwp_data(2005, 4, 2012, 12)
df = pd.DataFrame(data, columns=['year', 'day', 'dlink'])

df = convert_to_datetime(df)

df

,date,dlink
0,2005-04-07,https://www.bok.or.kr/portal/cmmn/file/fileDow...
1,2005-05-12,https://www.bok.or.kr/portal/cmmn/file/fileDow...
2,2005-06-09,https://www.bok.or.kr/portal/cmmn/file/fileDow...
3,2005-07-07,https://www.bok.or.kr/portal/cmmn/file/fileDow...
4,2005-08-11,https://www.bok.or.kr/portal/cmmn/file/fileDow...
...,...,...
89,2012-08-09,https://www.bok.or.kr/portal/cmmn/file/fileDow...
90,2012-09-13,https://www.bok.or.kr/portal/cmmn/file/fileDow...
91,2012-10-11,https://www.bok.or.kr/portal/cmmn/file/fileDow...
92,2012-11-09,https://www.bok.or.kr/portal/cmmn/file/fileDow...


In [4]:
#다운로드 파일에 저장하기

from selenium import webdriver
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()

for i in df['dlink']:
    driver.get(i)

time.sleep(20)

driver.quit()

In [5]:
#filename 리스트 만들기

import os

path = "C:/Users/kwkwo/Downloads/"
all_filenames = os.listdir(path)
hwp_files = []

for filename in all_filenames:
    if filename[-4:] == ".hwp":
        hwp_files.append(filename)

print(hwp_files)

['2005년+제10차+금통위+의사록.hwp', '2005년+제12차+금통위+의사록.hwp', '2005년+제14차+금통위+의사록.hwp', '2005년+제17차+금통위+의사록.hwp', '2005년+제19차+금통위+의사록.hwp', '2005년+제21차+금통위+의사록.hwp', '2005년+제24차+금통위+의사록.hwp', '2005년+제26차+금통위+의사록.hwp', 'munete20.hwp', '금융통화위원회+의사록(2006년도+제10차).hwp', '금융통화위원회+의사록(2006년도+제12차).hwp', '금융통화위원회+의사록(2006년도+제14차).hwp', '금융통화위원회+의사록(2006년도+제16차).hwp', '금융통화위원회+의사록(2006년도+제19차).hwp', '금융통화위원회+의사록(2006년도+제21차).hwp', '금융통화위원회+의사록(2006년도+제23차).hwp', '금융통화위원회+의사록(2006년도+제25차).hwp', '금융통화위원회+의사록(2006년도+제2차).hwp', '금융통화위원회+의사록(2006년도+제4차).hwp', '금융통화위원회+의사록(2006년도+제6차).hwp', '금융통화위원회+의사록(2006년도+제8차).hwp', '금융통화위원회+의사록(2007년도+제11차).hwp', '금융통화위원회+의사록(2007년도+제13차).hwp', '금융통화위원회+의사록(2007년도+제15차).HWP.hwp', '금융통화위원회+의사록(2007년도+제17차).hwp', '금융통화위원회+의사록(2007년도+제19차).hwp', '금융통화위원회+의사록(2007년도+제21차).hwp', '금융통화위원회+의사록(2007년도+제23차).hwp', '금융통화위원회+의사록(2007년도+제25차).hwp', '금융통화위원회+의사록(2007년도+제2차).hwp', '금융통화위원회+의사록(2007년도+제4차).HWP.hwp', '금융통화위원회+의사록(2007년도+제7차).hwp', '금융통화위원회+의사록(2007년도+제9차).HWP.hwp', '금융

In [15]:
import olefile
import zlib
import struct
import os
import re

# 다운로드 폴더 경로 (Windows에서는 기본적으로 사용자 폴더에 있음)
download_folder = os.path.join(os.path.expanduser("~"), "Downloads")

# .hwp 파일 목록 가져오기
hwp_files = [os.path.join(download_folder, f) for f in os.listdir(download_folder) if f.endswith('.hwp')]

# '통화정책방향〉' ~ '(４) 심의결과' 사이의 내용만 가져오는 함수
def extract_text_after_keyword(text):
    start_keyword = "통화정책방향〉"
    end_keyword = "(４) 심의결과"
    
    if start_keyword in text:
        # '통화정책방향〉' 이후의 텍스트만 추출
        extracted_text = text.split(start_keyword, 1)[1].strip()
        
        if end_keyword in extracted_text:
            # '(４) 심의결과' 이전의 텍스트만 반환
            extracted_text = extracted_text.split(end_keyword, 1)[0].strip()
        
        return extracted_text
    
    # 키워드가 없으면 빈 문자열 반환
    return ""

# '통화정책방향〉' 이전의 '0000년도 제00차 회의' 문자열을 가져오는 함수 <- title
def extract_text_before_keyword(text):
    keyword = "통화정책방향〉"
    
    if keyword in text:
        # '통화정책방향〉' 이전의 텍스트만 추출
        text_before_keyword = text.split(keyword, 1)[0]
        
        # 정규 표현식으로 '0000년도 제00차 회의' 패턴 찾기
        match = re.search(r'\d{4}년도 제\d{1,2}차 회의', text_before_keyword)
        
        if match:
            # 일치하는 패턴 반환
            return match.group(0)
    
    # 패턴이 없으면 빈 문자열 반환
    return ""

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()
    # print(dirs)

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
            ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        # 각 Section 내 text 추출
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

# 리스트 초기화
contents = []
meeting_titles = []

for hwp in hwp_files:
    # print(hwp)
    text = get_hwp_text(hwp)
    contents.append(text)

    # 첫 번째 '0000년도 제00차 회의' 형식의 문자열을 추출하여 리스트에 저장
    title = extract_text_before_keyword(text)
    if title:  # 만약 title이 None이 아닌 경우에만 추가
        meeting_titles.append(title)

# 텍스트 정제
extracted_contents = [extract_text_after_keyword(text) for text in contents]


In [17]:
df['title'] = meeting_titles
df['contents']=extracted_contents

df = df[['date', 'title', 'dlink', 'contents']]

2005년도 제12차 회의
('(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제68호 - ｢국내외 경제동향｣, 국제국장이 보고 제69호 - ｢외환․국제금융 '
 '동향｣, 그리고 금융시장국장이 보고 제70호 -｢금융시장 동향｣에 대하여 보고하였음(보고내용 : 별첨)\r\n'
 '(２) 본회의에서는 의장이 ｢한국은행법｣ 제28조에 의거하여 의안 제21호 - ｢통화정책방향｣을 상정하였음\r\n'
 '(３) 위원 토의내용\r\n'
 '   ｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와 소비재판매액 간에는 '
 '다소 괴리가 있을 수 있는 데다 소비와 밀접한 관련이 있는 서비스업활동지수 증가세가 확대되고 있는 점을 감안할 때 소비회복이 진행중인 '
 '것으로 보이며, 기계류 내수출하의 부진에도 불구하고 내수용 기계류수입이 호조를 보이고 있어 설비투자도 비관적으로 볼 상황은 아닌 것으로 '
 '생각된다는 견해를 나타내었음\r\n'
 '   동 위원은 서울 강남에서 시작된 부동산가격 상승이 다른 지역으로 확산될 기미가 있는 것으로 보인다며 아파트 거래량이 늘어나는 것은 '
 '추가적인 가격상승의 가능성이 높음을 시사하는 것이 아닌지 물었으며, 이에 대해 관련부서에서는 서울 강남과 판교 주변지역 등 일부 '
 '개발호재가 있는 지역을 중심으로 가격상승이 나타나고 있으나 서울 강북 등 여타 지역은 현재까지 별다른 영향을 받지 않고 있는 것으로 '
 '보이며 거래량도 계절성을 감안할 때 2002년이나 2003년에 비해 많은 수준은 아니라고 설명하였음\r\n'
 '   또한 동 위원은 정부가 여러 차례에 걸쳐 부동산시장 안정대책을 내놓았음에도 불구하고 오히려 정책의 대상이 되는 지역을 중심으로 '
 '가격상승이 지속되고 있어 규제위주의 부동산정책이 실효성이 없음을 보여주는 것으로 생각된다는 견해를 밝혔으며, 이에 대해 관련부서에서는 '
 '외부기관의 분석결과를 참조하는 한편 내부적으로 과거사례 등을 통해 분